In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
from tqdm import trange
import multiprocessing as mp
import seaborn as sns

from cvx.stat_arb.ccp import *
from cvx.stat_arb.utils import *
from cvx.stat_arb.backtest import *

sns.set()

%load_ext autoreload
%autoreload 2

In [2]:
naics_to_sector = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities', 
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade',
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

In [3]:
data = pd.read_csv("/Users/kasper/Documents/Stanford/Research/Data/CRSP/all_crsp_10_to_23.csv")
data.index = pd.to_datetime(data.date, format="%Y-%m-%d")
data = data.drop(columns=['date'])

prices = pd.pivot_table(data, values='PRC', index='date', columns='PERMCO', fill_value=None)
shares = pd.pivot_table(data, values='SHROUT', index='date', columns='PERMCO', fill_value=None)
market_cap = prices*shares

# Remove microcaps
# prices[market_cap < 10*1e6] = np.nan
# prices[market_cap > 0.5*1e9] = np.nan
prices = prices.dropna(axis=1, how='all', inplace=False)
# prices = prices.drop([540], axis=1) # Let's not trade BRK

prices_train = prices.loc["2022-01-01":"2022-07-31"].iloc[:,:]
prices_val = prices.loc["2022-08-01":"2022-10-31"].iloc[:,:]
prices_test = prices.loc["2022-10-01":"2022-12-31"].iloc[:,:]

prices_train_val = pd.concat([prices_train, prices_val]) # For later use

In [4]:
# data = pd.read_csv("/Users/kasper/Documents/Stanford/Research/Data/SP500/asset_data_00_to_23_w_industries.csv")
# data.index = pd.to_datetime(data.date, format="%Y-%m-%d")
# data = data.drop(columns=['date'])

# prices = pd.pivot_table(data, values='PRC', index='date', columns='PERMNO', fill_value=None)

# prices_train = prices.loc["2022-01-01":"2022-07-31"].iloc[:,:]
# prices_val = prices.loc["2022-08-01":"2022-10-31"].iloc[:,:]
# prices_test = prices.loc["2022-10-01":"2022-12-31"].iloc[:,:]

# prices_train_val = pd.concat([prices_train, prices_val]) # For later use

NAICS = pd.pivot_table(data, values='NAICS', index='date', columns='PERMCO',\
     fill_value=None)
NAICS = NAICS.fillna(method='ffill')
NAICS = NAICS.fillna(method='bfill')
NAICS = NAICS.applymap(lambda x: str(x)[:2]).astype(int)

# Get dict of sectors
asset_to_sector = {}
sector_to_asset = {}
assets_visited = []
for asset in NAICS.columns:
    if asset in assets_visited:
        print("fdsj")
    assets_visited.append(asset)
    if asset in prices.columns: # TODO: They have different columns...
        sectors_temp = []
        for n in NAICS[asset].unique():
            try:
                sector = naics_to_sector[str(n)]

                if sector not in sector_to_asset:
                    sector_to_asset[sector] = [asset]
                else:
                    if asset not in sector_to_asset[sector]:
                        sector_to_asset[sector].append(asset)
                sectors_temp.append(sector)

            except KeyError:
                print("Sector " + str(n) + " does not exist...")
        asset_to_sector[asset] = sectors_temp

Sector 10 does not exist...
Sector 67 does not exist...
Sector 99 does not exist...
Sector 99 does not exist...
Sector 99 does not exist...
Sector 99 does not exist...
Sector 99 does not exist...
Sector 99 does not exist...
Sector 67 does not exist...
Sector 73 does not exist...


In [15]:
all_prices_train[0].dropna(axis=1, how='all').shape

(144, 7077)

In [18]:
from cvx.stat_arb.ccp import *

np.random.seed(1)

P_max = 10*1e6
spread_max=1e6
n_candidates = 100

all_seeds = list(np.random.choice(range(10*n_candidates),\
     n_candidates, replace=False))

all_prices_train = [prices_train]*n_candidates
all_P_max = [P_max]*n_candidates
all_spread_max = [spread_max]*n_candidates
all_zero_inds = [None]*n_candidates
all_i = [i for i in range(n_candidates)]
all_n_candidates = [None]*n_candidates
all_p_init = [None]*n_candidates
all_s_init = [None]*n_candidates
all_mu_init = [None]*n_candidates


for i in trange(n_candidates):
     stat_arb = construct_stat_arb(all_prices_train[0].iloc[:,:500],\
               all_P_max[0], all_spread_max[0], all_zero_inds[0], all_i[0], all_n_candidates[0], all_p_init[0], all_s_init[0],\
                     all_mu_init[0], all_seeds[i])

 10%|█         | 10/100 [00:04<00:43,  2.06it/s]


IndexError: list index out of range

In [5]:
from cvx.stat_arb.ccp import *

np.random.seed(1)

P_max = 10
n_candidates = 10

all_seeds = list(np.random.choice(range(10*n_candidates),\
     n_candidates, replace=False))

all_prices_train = [prices_train]*n_candidates
all_P_max = [P_max]*n_candidates
all_zero_inds = [None]*n_candidates
all_i = [i for i in range(n_candidates)]
all_n_candidates = [n_candidates]*n_candidates
all_p_init = [None]*n_candidates
all_s_init = [None]*n_candidates
all_mu_init = [None]*n_candidates

sector_stat_arbs = {}
for i in trange(len(sector_to_asset)):
    sector = list(sector_to_asset.keys())[i]
    assets = sector_to_asset[sector]
    prices_sector = prices_train[assets]

    # Look for stat arbs if multiple assets in sector
    # TODO: should be len(assets) > 1?
    if len(prices_sector) > 1:
        all_prices_train = [prices_sector]*n_candidates
        pool = mp.Pool()
        all_stat_arbs = pool.starmap(construct_stat_arb, zip(all_prices_train,\
            all_P_max, all_zero_inds, all_i, all_n_candidates,\
                all_p_init, all_s_init, all_mu_init, all_seeds))
        pool.close()
        pool.join()
        sector_stat_arbs[sector] = all_stat_arbs
    

  0%|          | 0/20 [00:00<?, ?it/s]

0% 

  5%|▌         | 1/20 [00:10<03:15, 10.29s/it]

0% Solver failed...


 10%|█         | 2/20 [00:33<05:25, 18.07s/it]

...

 90%|█████████ | 18/20 [01:53<00:09,  4.74s/it]

0% 

 95%|█████████▌| 19/20 [01:55<00:04,  4.04s/it]

0% 

100%|██████████| 20/20 [01:57<00:00,  5.88s/it]


In [16]:
prices_train_new = prices_train.dropna(axis=1)

In [18]:
prices_train_new.isna().sum().sum()

0

In [24]:
prices_test[prices_train_new.columns].shape

(63, 6560)

# Trading backetests

In [13]:
all_portfolios = []

cutoff = 1.05
for sector in sector_stat_arbs.keys():
    # Get stat arbs
    all_stat_arbs = sector_stat_arbs[sector]
    traded_stat_arbs, non_traded_stat_arbs, total_profit \
        = filter_stat_arbs(all_stat_arbs, prices_train, prices_val, prices_test,\
        cutoff=cutoff, SR_cutoff=3)

    if len(traded_stat_arbs) > 0:
        # Create portfolios
        pos0 = traded_stat_arbs[0].get_positions(prices_test, cutoff=cutoff,\
            exit_last=True)
        portfolio = build_portfolio(prices_test, positions=pos0)

        for stat_arb in traded_stat_arbs[1:]:
            pos_temp = stat_arb.get_positions(prices_test, cutoff=cutoff, exit_last=True)
            portfolio += build_portfolio(prices_test, positions=pos_temp)
        

        
        print("\n", sector)
        print(f"# assets: {portfolio.positions.shape[1]}/{len(sector_to_asset[sector])}")
        m = portfolio.performance_metrics()
        print("profit: ", np.round(m.total_profit,1))
        print("SR: ", np.round(m.sr_profit,1))

        all_portfolios.append(portfolio)

[[ 4.62512512e-01]
 [ 2.21963497e-01]
 [ 1.07499464e-01]
 [ 2.25940469e-01]
 [ 1.36102996e-01]
 [-9.60830495e-02]
 [-9.25097387e-02]
 [-1.65547222e-01]
 [-2.97852361e-01]
 [-1.98695592e-01]
 [-2.72360827e-01]
 [-2.11383931e-01]
 [-5.21742549e-02]
 [-1.41693221e-01]
 [-1.95385071e-01]
 [-1.79176372e-01]
 [-2.02060067e-01]
 [-3.37225140e-01]
 [-2.83579882e-01]
 [-1.90337038e-01]
 [ 1.33283969e+00]
 [-2.32662221e-01]
 [-3.94514147e-01]
 [-1.83235628e-01]
 [-1.34987795e-01]
 [-2.24431487e-01]
 [-2.36278437e-01]
 [-4.03103846e-01]
 [-2.76807667e-01]
 [-4.57211354e-01]
 [-5.07166364e-01]
 [-6.55880232e-01]
 [-6.67160308e-01]
 [-5.81387301e-01]
 [-1.30757857e-01]
 [-2.25502434e-01]
 [ 6.37596379e-03]
 [-1.65194809e-01]
 [ 6.51929723e-02]
 [-1.44346604e-01]
 [-3.76150781e-01]
 [-4.19268721e-01]
 [-3.81323837e-01]
 [-1.36014287e-01]
 [-1.80470512e-01]
 [-1.42776668e-01]
 [-1.04587172e-03]
 [-3.84718542e-02]
 [-5.39955510e-02]
 [ 5.84387281e-02]
 [ 1.23015927e-01]
 [ 1.53494616e-01]
 [ 1.1211771

ValueError: Parameter value must be real.

In [24]:
# All sectors combined
portfolio = Portfolio()
n_assets_traded = 0
for p in all_portfolios:
    portfolio += p
    n_assets_traded += p.positions.shape[1]

print("\n", "All sectors")
print("# assets: ", n_assets_traded)
m = portfolio.performance_metrics()
print("profit: ", np.round(m.total_profit,1))
print("SR: ", np.round(m.sr_profit,1))


 All sectors
# assets:  36
profit:  42.8
SR:  2.6


In [28]:
portfolio.positions.shape

(63, 34)